In [1]:
import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

In [2]:
class AI(nn.Module):
    def __init__(self,conv_kernel):
        super(AI, self).__init__()
        self.conv_kernel = conv_kernel
        self.lstm_layers = 1
        self.lstm_depth = 1000
        self.cnn1 = nn.Conv2d(1,self.conv_kernel,5,stride=2,padding=1)
        self.cnn2 = nn.Conv2d(self.conv_kernel,self.conv_kernel,3,stride=2,padding=0)
        self.rnn = nn.LSTM(6 * 6, self.lstm_depth,self.lstm_layers)
        self.fc1 = nn.Linear(self.lstm_depth,self.lstm_depth)
        self.fc2 = nn.Linear(self.lstm_depth,10)
        self.out = nn.Dropout(0.9)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')     
        self.to(self.device)
    def forward(self,x):
        hx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        cx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn2(x))
#        print(x.shape)
        x = x.view(-1,self.conv_kernel,6 * 6)
        hx, cx = self.rnn(x, (hx, cx))
        o = T.tanh(self.fc1(hx))
        o = T.sigmoid(self.fc2(o))
#        o = self.out(o)
        return o

In [3]:
class PLAYFIELD():
    def __init__(self,width,height):
        self.width = width
        self.height = height
        self.pf = np.zeros((self.width,self.height,2))
    def setStone(self,x,y,stone):
        s = np.zeros(2)
        s[stone] = 1
        self.pf[y][x] = s
    def get(self):
        return self.pf

In [4]:
with open('train-images-idx3-ubyte.gz', 'rb') as f:
    train_images = extract_images(f)
with open('train-labels-idx1-ubyte.gz', 'rb') as f:
    train_labels = extract_labels(f)

with open('t10k-images-idx3-ubyte.gz', 'rb') as f:
    test_images = extract_images(f)
with open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
    test_labels = extract_labels(f)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz


In [5]:

ai_conv_kernel = 128
ai = AI(ai_conv_kernel)

max_mem = 1000
a = 28

inp = []
for i in range(max_mem):
    inp.append(np.random.randn(1, a, 28))
    
criterion = nn.MSELoss()#SmoothL1Loss()MSELoss
optimizer = optim.RMSprop(ai.parameters(), lr=0.00005)

losses = 0
mul = np.ones([1,28,10])
mul = T.from_numpy(mul).to(ai.device)
for j in range(1):
    for i in range(60000):#train_images.shape[0]):
        def closure():
            optimizer.zero_grad()
            inp1 = train_images[i]#inp[i % max_mem].copy() + np.random.randn(1, a, 28) * 0.1
            inp1.shape = (1,1,28,28)
            inp1 = T.from_numpy(inp1).float().to(ai.device)
            out = ai(inp1)
    #        print(out)
    #        print(target)
            target = out.cpu().detach().numpy().copy()
#            print(target.shape)
#            target[0][8 - 1] = np.zeros(10)
#            target[0][8 - 1][train_labels[i]] = 1.0

#            target[0] = np.zeros([1,8,10])
            n = np.zeros(10)
            n[train_labels[i]] = 1.0
            for o in range(ai_conv_kernel):
                oo = (o + 1) / ai_conv_kernel
                oo = oo * oo
                target[0][o] = oo * n + (1.0 - oo) * target[0][o]
#            target[0][8 - 1][train_labels[i]] = 1.0
#            print(target)
            target = T.from_numpy(np.array(target)).to(ai.device)
            loss = criterion(out, target.float())
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        losses = losses + loss.item()
        if (i % 100) == 0:
            print('Step: ' + str(i) + ' loss: ' + str(losses / 100))
            losses = 0

Step: 0 loss: 0.0005046569555997849
Step: 100 loss: 0.01817169249523431
Step: 200 loss: 0.012863098946399986
Step: 300 loss: 0.009757395560154692
Step: 400 loss: 0.007608264443697408
Step: 500 loss: 0.007961103776469827
Step: 600 loss: 0.00827303237048909
Step: 700 loss: 0.007086562535841949
Step: 800 loss: 0.006514218118391
Step: 900 loss: 0.006338046362507157
Step: 1000 loss: 0.005819015542947455
Step: 1100 loss: 0.006867442991642747
Step: 1200 loss: 0.005731767673860304
Step: 1300 loss: 0.006285862107179127
Step: 1400 loss: 0.004916772861324716
Step: 1500 loss: 0.004321016228204826
Step: 1600 loss: 0.004474301819282118
Step: 1700 loss: 0.003612013930542162
Step: 1800 loss: 0.0028671061703062152
Step: 1900 loss: 0.003611241973412689
Step: 2000 loss: 0.0041458495514962125
Step: 2100 loss: 0.004493254055851139
Step: 2200 loss: 0.002982811086840229
Step: 2300 loss: 0.003472946363981464
Step: 2400 loss: 0.0033579270273185104
Step: 2500 loss: 0.0034080125753825994
Step: 2600 loss: 0.00291

Step: 21100 loss: 0.0018355934805939001
Step: 21200 loss: 0.0015983436374904158
Step: 21300 loss: 0.002020712752192253
Step: 21400 loss: 0.002117808804443655
Step: 21500 loss: 0.0017531432498208233
Step: 21600 loss: 0.0016013273025333773
Step: 21700 loss: 0.0015350043096643163
Step: 21800 loss: 0.0015093921632222873
Step: 21900 loss: 0.0009767528822999339
Step: 22000 loss: 0.0021328365136309914
Step: 22100 loss: 0.0011789420455821187
Step: 22200 loss: 0.0029671033096170165
Step: 22300 loss: 0.00268715617099474
Step: 22400 loss: 0.0014626959532762385
Step: 22500 loss: 0.003239534650428766
Step: 22600 loss: 0.0036299828219563323
Step: 22700 loss: 0.0023891386762159073
Step: 22800 loss: 0.0019422320698873818
Step: 22900 loss: 0.0021722476890488453
Step: 23000 loss: 0.0002726817906741985
Step: 23100 loss: 0.0018496547346299507
Step: 23200 loss: 0.0015528671749699186
Step: 23300 loss: 0.0018695671261949086
Step: 23400 loss: 0.0017287999813329691
Step: 23500 loss: 0.0017825345905009727
Step:

Step: 41800 loss: 0.001050344620508028
Step: 41900 loss: 0.0008869766763665155
Step: 42000 loss: 0.0021347875613574275
Step: 42100 loss: 0.0014563210960925944
Step: 42200 loss: 0.0016212739999935622
Step: 42300 loss: 0.0006336527117355217
Step: 42400 loss: 0.0018524944721953318
Step: 42500 loss: 0.0017946608329594404
Step: 42600 loss: 0.0017285649855168827
Step: 42700 loss: 0.0021286509376960082
Step: 42800 loss: 0.0015023566651799314
Step: 42900 loss: 0.002077143454157735
Step: 43000 loss: 0.002937463220169789
Step: 43100 loss: 0.001862028916157179
Step: 43200 loss: 0.0015413248099489963
Step: 43300 loss: 0.001576594021845743
Step: 43400 loss: 0.00016940640480353153
Step: 43500 loss: 0.001250372108179718
Step: 43600 loss: 0.0019007570957739262
Step: 43700 loss: 0.0015516138296054027
Step: 43800 loss: 0.0007665712650778111
Step: 43900 loss: 0.0009085928822441503
Step: 44000 loss: 0.0014063388162635527
Step: 44100 loss: 0.0011683278786968998
Step: 44200 loss: 0.0014506484261334763
Step:

In [6]:
count = 0
for i in range(10000):#train_images.shape[0]):
    optimizer.zero_grad()
    inp1 = test_images[i]#inp[i % max_mem].copy()
    inp1.shape = (1,1,28,28)
    inp1 = T.from_numpy(inp1).float().to(ai.device)
    out = ai(inp1).cpu().detach().numpy()
    m = np.argmax(out[0][5 - 1])
    if m == test_labels[i]:
#        print(m)
        count = count + 1
print("correct: " + str(count / 10000))#train_images.shape[0]))

correct: 0.9512
